In [ ]:
import uproot
import pandas as pd
import matplotlib.pyplot as plt
import awkward as ak

tables = [
    "O2hfcredcorrpair",
    "O2hfcredhadhad",
]

with uproot.open("/home/mdicosta/LocalTestsO2/CorrelationsOO/CorrelationTables/HadHad/MixedEvent/AO2D.root") as f:
    keys = f.keys()
    print("ROOT Keys:", keys)
    
    for table in tables:
        # Filter keys that match the table name
        matched_keys = [key for key in keys if table in key]
        
        if not matched_keys:
            print(f"No keys found for table: {table}")
            continue
        
        dfs = []
        for key in matched_keys:
            print(f"Reading {key}")
            array = f[key].arrays(library="ak")  # get awkward array
            df = ak.to_dataframe(array)          # convert to pandas DataFrame
            dfs.append(df)
        
        full_df = pd.concat(dfs, ignore_index=True)
        print(f"\nTable: {table}")
        print(f"Number of entries: {len(full_df)}")
        print(f"Columns: {full_df.columns.tolist()}")
        
        # Plotting
        n_columns = len(full_df.columns)
        n_rows = (n_columns // 4) + (1 if n_columns % 4 else 0)
        
        fig, axes = plt.subplots(n_rows, 4, figsize=(15, 5 * n_rows))
        axes = axes.flatten()

        for i, column in enumerate(full_df.columns):
            print(f"Plotting {column}")
            try:
                if 'Centrality' in column:
                    axes[i].hist(full_df[column].dropna(), bins=20, color='blue')
                else:
                    axes[i].hist(full_df[column].dropna(), bins=50, color='blue')
                axes[i].set_title(f'{column}')
                axes[i].set_xlabel('Value')
                axes[i].set_ylabel('Frequency')
                # axes[i].set_yscale('log')
            except Exception as e:
                print(f"Could not plot column {column}: {e}")
                axes[i].set_visible(False)
        
        # Remove unused axes if any
        for i in range(n_columns, len(axes)):
            fig.delaxes(axes[i])
        
        fig.suptitle(f"Histograms for table: {table}", fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()


ROOT Keys: ['DF_2424272862165760;1', 'DF_2424272862165760/O2hfcredchhad;1', 'DF_2424272862165760/O2hfcredhadhad;1', 'parentFiles;1']
No keys found for table: O2hfcredcorrpair
